In [ ]:
class Strategy:
    def __init__(self):
        self.strategy = "Base strategy"

    def decide_to_buy(self, player, space):
        # TO DO: MAKE THIS REFLECT A BASE STRATEGY FOR BUYING/NOT BUYING A PROPERTY
        # TO DO: ADD OPTION TO MORTGAGE/SELL HOUSES TO PURCHASE A PROPERTY
        if player.money >= space.price:

            # buy the property if they already have one in that set OR if they have 2 or fewer properties in total
            if space.type == "Street":
                if len(player.property_sets[space.group]) > 0:
                    return True
                elif len(player.properties) < 3:
                    return True
                else:
                    return False
                
            elif space.type == "Station":
                return True
            elif space.type == "Utility":
                return True
            else:
                return False

        else:
            return False
        
    def decide_sell_houses(self, player, money_needed):
        # player's properties (streets) with houses or hotels
        properties_with_houses = [prop for prop in player.properties if prop.num_houses > 0]

        # sort properties with houses by their house count (most houses first)
        properties_with_houses.sort(key = lambda prop: prop.num_houses + prop.hotel, reverse=True)

        money_raised = 0

        # iterate through properties with houses and sell houses until enough money is raised
        for prop in properties_with_houses:

            house_sale_value = prop.calculate_house_sale_value()

            # determine number of houses to sell on this property
            houses_to_sell = int(min(prop.num_houses+prop.hotel, (money_needed // house_sale_value)+1))

            if houses_to_sell > 0:
                total_sale_value = houses_to_sell * house_sale_value

                if prop.hotel:
                    prop.hotel = False
                    player.hotels -= 1
                    prop.num_houses -= (houses_to_sell - 1)
                    player.houses -= (houses_to_sell - 1)
                else:
                    prop.num_houses -= houses_to_sell
                    player.houses -= houses_to_sell

                player.money += total_sale_value
                money_raised += total_sale_value

                # check if enough money has been raised to meet the target
                if money_raised >= money_needed:
                    break

    def decide_mortgage_properties(self, player, money_needed):
        # combine all of the player's properties (streets, stations, and utilities) without buildings
        undeveloped_streets = [prop for prop in player.properties if prop.num_houses == 0]
        all_properties = undeveloped_streets + player.stations + player.utilities

        # sort combined list of properties by mortgage value (least valuable first)
        all_properties.sort(key = lambda prop: prop.calculate_mortgage_value(), reverse=False)

        money_raised = 0

        # iterate through the player's properties and mortgage them until enough money is raised
        for prop in all_properties:
            if not prop.is_mortgaged:
                
                mortgage_value = prop.calculate_mortgage_value()

                prop.is_mortgaged = True
                money_raised += mortgage_value
                player.money += mortgage_value

            # check if enough money has been raised to meet the target
            if money_raised >= money_needed:
                break

    def decide_to_leave_jail(self, player):
        # TO DO: MAKE THIS REFLECT BASE STRATEGY FOR LEAVING/NOT LEAVING JAIL
        # TO DO: ADD OPTION TO MORTGAGE/SELL HOUSES TO LEAVE JAIL
        if player.jail_cards > 0:
            return True
        elif player.money >= 50:
            player.pay(50)
            return True
        else:
            return False
        
    def decide_build_on_properties(self, player, property_sets):
        # look through all property groups
        for group, properties in property_sets.items():
            player_properties = player.property_sets[group]

            # determine if the entire set is owned
            if sorted(properties) == sorted(player_properties):
                for street in properties:
                    
                    # decision to develop property or not
                    if self.decide_to_build_house(player, street):
                        player.pay(street.house_price)
                        street.num_houses += 1
                        player.houses += 1
                    elif self.decide_to_build_hotel(player, street):
                        player.pay(street.house_price)
                        street.hotel = True
                        player.hotels += 1
                    else:
                        pass

    def decide_to_build_house(self, player, street):
        # TO DO: MAKE THIS REFLECT BASE STRATEGY FOR BUILDING ON PROPERTIES
        # TO DO: ADD OPTION TO MORTGAGE/SELL HOUSES TO BUILD ON PROPERTIES

        if street.num_houses < 4 and player.money >= street.house_price:
            # get neighboring properties in the same set
            neighbouring_properties = [prop for prop in player.property_sets[street.group] if prop != street]

            # check if there is no greater than 1 house difference on neighboring properties
            for neighbour_property in neighbouring_properties:
                if abs(street.num_houses - neighbour_property.num_houses) > 1:
                    return False 

            return True 
        
        else:
            return False
        
    def decide_to_build_hotel(self, player, street):
        # TO DO: MAKE THIS REFLECT BASE STRATEGY FOR BUILDING ON PROPERTIES
        # TO DO: ADD OPTION TO MORTGAGE/SELL HOUSES TO BUILD HOTELS

        if street.num_houses == 4 and not street.hotel and player.money >= street.house_price:
            # get neighboring properties in the same set
            neighbouring_properties = [prop for prop in player.property_sets[street.group] if prop != street]

            # check if there is no greater than 1 house difference on neighboring properties
            for neighbour_property in neighbouring_properties:
                if abs(street.num_houses + street.hotel - neighbour_property.num_houses - neighbour_property.hotel) > 1:
                    return False  

            return True 
        
        else:
            return False